In [4]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql.functions import row_number, col
from pyspark.sql.window import Window


StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 6, Finished, Available)

In [2]:
df=spark.read.format('parquet').load('Files/Cleaned_Data')

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 4, Finished, Available)

### Area Dimension


In [5]:
window=Window.orderBy(col('BOROUGH'),col('LATITUDE'),col('LONGITUDE'),col('ZIP CODE'))
 
area_df=df.select(col('BOROUGH'),col('LATITUDE'),col('LONGITUDE'),col('ZIP CODE')).distinct()
area_df=area_df.withColumn("Area_ID",row_number().over(window))
area_df.write.mode("overwrite").parquet("Files/Dimension_model/area_dim")

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 7, Finished, Available)

### Time Dimension

In [10]:
time_dim=df.select(col('CRASH TIME'))
time_dim = time_dim.withColumn('hour', hour('CRASH TIME')).withColumn('minute', minute('CRASH TIME'))

time_df=time_dim.distinct()
window=Window.orderBy(col('CRASH TIME'))
time_df=time_df.withColumn("time_ID",row_number().over(window))

time_df = time_df.withColumn(
    "hour", 
    when(time_df["CRASH TIME"] == "0", 0).otherwise(time_df["hour"])
).withColumn(
    "minute",
    when(time_df["CRASH TIME"] == "0", 0).otherwise(time_df["minute"])
)
time_df.write.mode("overwrite").parquet("Files/Dimension_model/time_dim")

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 12, Finished, Available)

### Vehicle_Dimension

In [16]:
vehicle_df=df.select(col('COLLISION_ID'),col('VEHICLE TYPE CODE 1'),col('VEHICLE TYPE CODE 2'),col('VEHICLE TYPE CODE 3'),col('VEHICLE TYPE CODE 4'),col('VEHICLE TYPE CODE 5'))
vehicle_df.createOrReplaceTempView("vehicle_df")

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 18, Finished, Available)

In [17]:
vehicle_df = spark.sql("""
    SELECT COLLISION_ID,
       `VEHICLE TYPE CODE 1`
FROM vehicle_df

union all

SELECT COLLISION_ID,
       `VEHICLE TYPE CODE 2`
FROM vehicle_df

union all

SELECT COLLISION_ID,
       `VEHICLE TYPE CODE 3`
FROM vehicle_df

union all

SELECT COLLISION_ID,
       `VEHICLE TYPE CODE 4`
FROM vehicle_df

union all

SELECT COLLISION_ID,
       `VEHICLE TYPE CODE 5`
FROM vehicle_df

""")

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 19, Finished, Available)

In [18]:
# Selecting columns and aliasing 'VEHICLE TYPE CODE 1' as 'vehicle_type'
vehicle_collision = vehicle_df.select(col('COLLISION_ID'), col('VEHICLE TYPE CODE 1').alias('vehicle_type'))

# Filtering rows where 'vehicle_type' is null
vehicle_collision = vehicle_collision.filter(col('vehicle_type').isNotNull())

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 20, Finished, Available)

In [19]:
vehicle_dim=vehicle_collision.select(col('vehicle_type')).distinct()
window=Window.orderBy(col('vehicle_type'))
vehicle_dim=vehicle_dim.withColumn("vehicle_id",row_number().over(window))
vehicle_dim.write.mode("overwrite").parquet("Files/Dimension_model/vehicle_dim")

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 21, Finished, Available)

### Vehicle_collision


In [20]:
vehicle_collision = vehicle_dim.join(vehicle_collision, on='vehicle_type', how='inner')
vehicle_collision=vehicle_collision.select(col('vehicle_id'),col('COLLISION_ID'))
vehicle_collision.write.mode("overwrite").parquet("Files/Dimension_model/vehicle_collision")

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 22, Finished, Available)

### Contribution Dimension 

In [24]:
contribution_df=df.select(col('COLLISION_ID'),col('CONTRIBUTING FACTOR VEHICLE 1'),col('CONTRIBUTING FACTOR VEHICLE 2'),col('CONTRIBUTING FACTOR VEHICLE 3'),col('CONTRIBUTING FACTOR VEHICLE 4'),col('CONTRIBUTING FACTOR VEHICLE 5'))

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 26, Finished, Available)

In [25]:
contribution_df.createOrReplaceTempView("contribution_df")
contribution_collision = spark.sql("""
    SELECT COLLISION_ID,
       `CONTRIBUTING FACTOR VEHICLE 1`
FROM contribution_df

union all

SELECT COLLISION_ID,
       `CONTRIBUTING FACTOR VEHICLE 2`
FROM contribution_df

union all

SELECT COLLISION_ID,
       `CONTRIBUTING FACTOR VEHICLE 3`
FROM contribution_df

union all

SELECT COLLISION_ID,
       `CONTRIBUTING FACTOR VEHICLE 4`
FROM contribution_df

union all

SELECT COLLISION_ID,
       `CONTRIBUTING FACTOR VEHICLE 5`
FROM contribution_df
""")

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 27, Finished, Available)

In [26]:
contribution_collision=contribution_collision.select(col('COLLISION_ID'),col('CONTRIBUTING FACTOR VEHICLE 1').alias('contribution_factor'))
contribution_collision = contribution_collision.filter(col('contribution_factor').isNotNull())

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 28, Finished, Available)

In [27]:
contribution_dim=contribution_collision.select(col('contribution_factor')).distinct()
window=Window.orderBy(col('contribution_factor'))
contribution_dim=contribution_dim.withColumn("contribution_id",row_number().over(window))
contribution_dim.write.mode("overwrite").parquet("Files/Dimension_model/contribution_dim")

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 29, Finished, Available)

### Contribution Collision

In [28]:
contribution_collision = contribution_dim.join(contribution_collision, on='contribution_factor', how='inner')
contribution_collision=contribution_collision.select(col('COLLISION_ID'),col('contribution_id'))
contribution_collision.write.mode("overwrite").parquet("Files/Dimension_model/contribution_collision")

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 30, Finished, Available)

### Date Dimension

In [29]:
date_dim=df.select(col('CRASH DATE'))

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 31, Finished, Available)

In [55]:
from pyspark.sql import functions as F

date_dim = df.select(F.to_date(F.col('CRASH DATE'), 'MM/dd/yyyy').alias('CRASH_DATE'))
date_dim=date_dim.distinct()

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 57, Finished, Available)

In [63]:
date_dim.createOrReplaceTempView("date_dim")
date_dim=spark.sql("""
    SELECT replace(`CRASH_DATE`,"-","") Date_id,
           `CRASH_DATE`,
           day(CRASH_DATE) Day ,
           month(CRASH_DATE) Month,
           year(CRASH_DATE) Year,
           Date_format(CRASH_DATE,'EEEE') Day_of_week,
           case When Date_format(CRASH_DATE,'EEEE') in ("Sunday","Saturday") then 1 else 0 end is_weekend,
           case when Month(CRASH_DATE) Between 1 and 4 then "Spring"
                when MOnth(CRASH_DATE) Between 5 and 8 then "Summer"
                else "Fall" end Season
           from date_dim
""")

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 65, Finished, Available)

In [65]:
from pyspark.sql import functions as F

date_dim = date_dim.withColumn("Date_id",
                             F.when(F.col("Date_id").isNull(), "0")
                             .otherwise(F.col("Date_id")))


StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 67, Finished, Available)

In [66]:
from pyspark.sql import functions as F

# Filter rows where 'CRASH_DATE' is null and display the results
date_dim.filter(F.col("CRASH_DATE").isNull()).show()


StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 68, Finished, Available)

+-------+----------+----+-----+----+-----------+----------+------+
|Date_id|CRASH_DATE| Day|Month|Year|Day_of_week|is_weekend|Season|
+-------+----------+----+-----+----+-----------+----------+------+
|      0|      null|null| null|null|       null|         0|  Fall|
+-------+----------+----+-----+----+-----------+----------+------+



In [67]:
date_dim.write.mode("overwrite").parquet("Files/Dimension_model/date_dim")

StatementMeta(, 175e4ff0-3078-47c5-82ac-f16fa08ed4ee, 69, Finished, Available)